In [ ]:
import os
model_dir = "A:/python_ai_projects/tool_tracking/tf-ssd"
model_name = "ssd_test"
sample_test_dir = "D:/python_ai_projects/tool_segmentation/sample_data"

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Conv2D, Input, InputLayer, BatchNormalization

w = 10
h = 10
ch = 4
stride = 1
# Define a simple Keras model
inputs = Input((h, w, ch), name=model_name)

channels = 8
x = Conv2D(channels,  (1, 1), padding='same', strides=(stride, stride), name="Conv1")(inputs)
output = BatchNormalization(name="bn")(x)
model = Model(inputs, output)

# Get the weights of the first layer
layer1_weights = model.layers[1].get_weights()
layer2_weights = model.layers[2].get_weights()

# Create a new set of weights with the same shape as the original weights
layer1_kernel_weight = np.random.uniform(low=0.0, high=2.0, size = layer1_weights[0].shape).astype(np.int32).astype(np.float32)
layer1_bias = np.array([0 if x < 2 else 0 for x in range(channels)])
new_layer1_weights = [layer1_kernel_weight, layer1_bias]
print(new_layer1_weights[0].shape, new_layer1_weights[1].shape)

gamma = np.ones(layer2_weights[0].shape)
beta = np.zeros(layer2_weights[1].shape)
moving_mean = np.zeros(layer2_weights[2].shape)
moving_variance = np.ones(layer2_weights[3].shape)
new_layer2_weights = [gamma, beta, moving_mean, moving_variance]

if layer1_kernel_weight.shape[0] == layer1_kernel_weight.shape[1] and layer1_kernel_weight.shape[0] == 1 and len(layer1_kernel_weight.shape) == 4:
    for i in range(min(7, layer1_kernel_weight.shape[3])):
        for j in range(min(7, layer1_kernel_weight.shape[2])):     
            print(layer1_kernel_weight[0,0,j,i], end=", ")   
        print("\n")
else:       
    for l in range(16):
        for k in range(2):
            for i in range(3):
                for j in range(3):
                    if l > 2 or i != j:
                        layer1_kernel_weight[i, j, k ,l] = 0
                    else:
                        layer1_kernel_weight[i, j, k ,l] = 1
                    print(layer1_kernel_weight[j, i, k ,l], end=", ")
                print()
            print()
        print("---------")
    
print(new_layer2_weights, len(new_layer2_weights))
# Set the weights of the first layer to the new weights
print(model.layers[1].name)
print(new_layer1_weights)
model.layers[1].set_weights(new_layer1_weights)
print(model.layers[2].name)
model.layers[2].set_weights(new_layer2_weights)


def pred_funct(img_data, model_p):
    img_in = np.expand_dims(img_data, axis = 0)
    return model_p.predict(img_in)

In [ ]:
import numpy as np
from keras.models import Model
from keras.layers import Conv2D, Input, InputLayer, BatchNormalization, Conv2DTranspose


w = 6
h = 4
ch = 16
stride = 2
# Define a simple Keras model
inputs = Input((h, w, ch), name=model_name)

channels = 16
output = Conv2DTranspose(channels,  (2, 2), padding='same', strides=(stride, stride), use_bias = False, name="conv2d_transpose")(inputs)
model = Model(inputs, output)

# Get the weights of the first layer
layer1_weights = model.layers[1].get_weights()
print("Length : ", len(layer1_weights))
print("Length : ", len(layer1_weights))

# Create a new set of weights with the same shape as the original weights
layer1_kernel_weight = np.random.uniform(low=0.0, high=2.0, size = layer1_weights[0].shape).astype(np.float32)
new_layer1_weights = [layer1_kernel_weight]

   
# Set the weights of the first layer to the new weights
print(model.layers[1].name)
model.layers[1].set_weights(new_layer1_weights)


def pred_funct(img_data, model_p):
    img_in = np.expand_dims(img_data, axis = 0)
    return model_p.predict(img_in)

In [ ]:
img_data = np.random.uniform(low=0.0, high=5.0, size = model.layers[0].input.shape[1:4]).astype(np.float32)
numpy_in = os.path.join(sample_test_dir, "in_" + model_name + ".npy")
np.save(numpy_in, np.transpose(img_data,[1,0,2]))
print(numpy_in)
print(img_data.shape)

for k in range(3):
    for i in range(4):
        for j in range(6):
            print(img_data[i, j, k], end=", ")
        print()
    print()

In [ ]:
output = pred_funct(img_data, model).squeeze()
print(img_data.shape)
print(output.shape) 
print("-----------------")
for k in range(6):
    for i in range(4):
        for j in range(6):
            print(img_data[i, j, k], end=", ")
        print()
    print("\n")
print("-----------------")
for k in range(6): 
    for i in range(min(8,output.shape[0])):
        for j in range(min(6,output.shape[1])):
            print(output[i, j, k], end=", ")
        print()
    print("\n") 

In [ ]:
import numpy as np
import os
from tensorflow.keras.layers import Activation, Concatenate, Add, ZeroPadding2D, BatchNormalization, Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, InputLayer, Conv2DTranspose, DepthwiseConv2D

epsilon = np.finfo(float).eps
lines_in_graph = []

model_data_dir = os.path.join(model_dir, model_name)
if not os.path.exists(model_data_dir):
    os.makedirs(model_data_dir)
    
for i, layer in enumerate(model.layers): 
    print(i, ":", layer.__class__.__name__, layer.name)
    items_to_write = [layer.__class__.__name__, layer.name]
    if isinstance(layer,InputLayer):
        inpu_shape = layer.input.shape
        items_to_write.append("")
        items_to_write.append(str(inpu_shape[2]))
        items_to_write.append(str(inpu_shape[1]))
        items_to_write.append(str(inpu_shape[3]))
    elif isinstance(layer.input, list):
        input_layer_names = []
        for i in range(len(layer.input)):
            inpu_layer_name = layer.input[i].name.split('/')[0]
            input_layer_names.append(inpu_layer_name)
        items_to_write.append("&".join(input_layer_names))
        if isinstance(layer,Add):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,Concatenate): 
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,HeadWrapper):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        else:
            print("---------------------------------")
            print("Error: Multi input layer not known")
            print("---------------------------------")            
    else:
        inpu_layer_name = layer.input.name.split('/')[0]
        items_to_write.append(inpu_layer_name)
        if isinstance(layer,Conv2D):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            dilation_rates = layer.dilation_rate
            if dilation_rates[0] != dilation_rates[1] and dilation_rates[1] != 1:
                print("---------------------------------")
                print("Error: dilation_rates must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,Conv2DTranspose):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,DepthwiseConv2D):   
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,BatchNormalization):
            weights = layer.get_weights()
            gamma = weights[0]
            beta = weights[1]
            moving_mean = weights[2]
            moving_variance = weights[3]
            a = gamma / np.sqrt(moving_variance + epsilon)
            b = - moving_mean * a + beta
            path_to_save = os.path.join(model_data_dir, layer.name + '_mean.npy')
            np.save(path_to_save, b)
            path_to_save = os.path.join(model_data_dir, layer.name + '_variance.npy')
            np.save(path_to_save, a) 

            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,MaxPooling2D):
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,ZeroPadding2D):
            if(layer.padding[0][1] != layer.padding[1][1] and layer.padding[0][0] == 0 and layer.padding[1][0] != 0):
                print(layer.padding)
                print("---------------------------------")
                print("Error: This padding not supported") 
                print("---------------------------------")
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(layer.padding[0][0]))
            items_to_write.append(str(layer.padding[0][1]))
            items_to_write.append(str(layer.padding[1][0]))
            items_to_write.append(str(layer.padding[1][1]))
        elif isinstance(layer, ReLU):
            maxValue = layer.max_value
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(maxValue))
        elif isinstance(layer, Activation):
            activation_type = layer.get_config()['activation']
            maxValue = 0
            if activation_type == 'relu':
                items_to_write.append("relu")
                maxValue = 0
                if hasattr(layer,'max_value'):
                    maxValue = layer.max_value
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
                items_to_write.append(str(maxValue))
            elif activation_type == 'sigmoid':
                items_to_write.append("sigmoid")
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
            elif activation_type == 'softmax':
                items_to_write.append("softmax")
                inpu_shape = layer.input.shape
                output_shape = layer.output.shape
                print(output_shape)
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[2]))
            else:
                print("---------------------------------")
                print("Error: Only Relu activation with max value supported, provided is ", activation_type)
                print("---------------------------------") 
        else:
            print("Error: Layer is not identified ", layer.__class__.__name__, layer.name)
    lines_in_graph.append(",".join(items_to_write))
with open(f'{model_dir}/{model_name}.txt', 'w') as f:
    # Write each line of the list to the file
    for line in lines_in_graph:            
        f.write(line + '\n')
    print("File is saved to " + f'{model_name}.txt')    

In [ ]:
np.set_printoptions(precision=5, suppress=True)
filer_name = "Conv1"
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_mean.npy')
if os.path.exists(path_to_saved):
    data = np.load(path_to_saved)
    for i in range(5):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_variance.npy')
    data = np.load(path_to_saved)
    for i in range(5):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
else:
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_bias.npy')
    data = np.load(path_to_saved)
    for i in range(5):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_weights.npy')
data = np.load(path_to_saved)

for l in range(2):
    for k in range(2):
        for i in range(3):
            for j in range(3):
                print(data[j, i, k ,l], end=", ")
            print()
        print("")
    print("+++++++++")
print(data.shape)
print("\n--------------")

In [ ]:
np.set_printoptions(precision=5, suppress=True)
filer_name = "block5_conv3"
path_to_saved = os.path.join(model_data_dir, f'{filer_name}.npy')
data = np.load(path_to_saved)

print(data.shape)
for l in range(2):
    for k in range(16):
        for i in range(2):
            for j in range(2):
                print(data[j, i, k ,l], end=", ")
            print()
        print("")
    print("+++++++++")
print("\n--------------")

In [ ]:
for index, layer in enumerate(model.layers):
    if "activation_8" == layer.name:
        print(index)     

In [ ]:
input_layer_data = model.layers[0]
intermediate_layer_data = model.layers[161]
output_layer_data = model.layers[162]

model1 = Model(input_layer_data.input, intermediate_layer_data.output)
model2 = Model(intermediate_layer_data.output, output_layer_data.output)

print(input_layer_data.name)
print(intermediate_layer_data.name)
print(output_layer_data.name)

print(input_layer_data.input.shape[1:])
print(intermediate_layer_data.output.shape[1:])
print(output_layer_data.output.shape[1:])

In [ ]:
intermediate_output = pred_funct(img_data, model1).squeeze()
output = pred_funct(intermediate_output, model2).squeeze()
print("-----------------")
for k in range(3):
    for i in range(6):
        for j in range(6):
            print(img_data[i, j, k], end=", ")
        print()
    print("\n")
print(img_data.shape)
print("-----------------")
for k in range(min(3, intermediate_output.shape[2])): 
    for i in range(6):
        for j in range(6):
            print(intermediate_output[i, j, k], end=", ")
        print()
    print("\n")
print(intermediate_output.shape)  
print("-----------------")
for k in range(min(3, output.shape[2])): 
    for i in range(6):
        for j in range(6):
            print(output[i, j, k], end=", ")
        print()
    print("\n")
print(output.shape)    

In [ ]:
def write_csv3d(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, output_layer_data.name + "_" + file_name_sufix +"_python.csv")
    print(output_data.shape)
    # Loop over the rows and columns of the 3D array
    with open(path_to_saved, 'w') as file:
        for l in range(output_data.shape[3]):
            for k in range(output_data.shape[2]):
                for j in range(output_data.shape[1]):
                    for i in range(output_data.shape[0]):
                        # Join the text data in each row with commas
                        value = output_data[i, j, k, l]
                        # Write the row data to the output file
                        if i == output_data.shape[1] - 1:
                            file.write("{:.4f}".format(value))
                        else:
                            file.write("{:.4f},".format(value))
                    file.write("\n")
                file.write("\n")
            file.write("\n")   
            
def write_csv(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, output_layer_data.name + "_" + file_name_sufix +"_python.csv")
    print(output_data.shape)
    # Loop over the rows and columns of the 3D array
    with open(path_to_saved, 'w') as file:
        for k in range(output_data.shape[2]):
            for j in range(output_data.shape[0]):
                for i in range(output_data.shape[1]):
                    # Join the text data in each row with commas
                    value = output_data[j, i, k]
                    # Write the row data to the output file
                    if i == output_data.shape[1] - 1:
                        file.write("{:.4f}".format(value))
                    else:
                        file.write("{:.4f},".format(value))
                file.write("\n")
            file.write("\n")       
            
write_csv(output, "out")
write_csv(intermediate_output, "in")         

In [ ]:
img_data = np.random.uniform(low=0.0, high=10.0, size = input_layer_data.input.shape[1:]).astype(np.int32).astype(np.float32)
#img_data = np.ones(input_layer_data.input.shape[1:], dtype=np.float32)
#img_data = np.zeros(input_layer_data.input.shape[1:], dtype=np.float32)

In [ ]:
np.set_printoptions(precision=5, suppress=True)
filer_name = "Conv1"
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_mean.npy')
if os.path.exists(path_to_saved):
    data = np.load(path_to_saved)
    for i in range(5):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_variance.npy')
    data = np.load(path_to_saved)
    for i in range(5):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
else:
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_bias.npy')
    data = np.load(path_to_saved)
    for i in range(5):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_weights.npy')
data = np.load(path_to_saved)
if 'depth' in filer_name:
    for i in range(3):
        for j in range(3):
            print(data[i, j, 0, 0], end=", ")
        print()
    print()
    for i in range(3):
        for j in range(3):
            print(data[i, j, 1, 0], end=", ")
        print()
else:
    for i in range(3):
        for j in range(3):
            print(data[0, 0, i, j], end=", ")
        print()
print(data.shape)
print("\n--------------")

In [ ]:
#filer_name = "expanded_conv_project"
filer_name = "Conv1"
numpy_in = os.path.join(sample_test_dir, filer_name + "_in.npy")
np.save(numpy_in, np.transpose(img_data,[1,0,2]))
print(numpy_in)

In [ ]:
file_name = "conv2d_transpose"
path_to_saved = os.path.join(model_data_dir, f'{file_name}.npy')
data = np.load(path_to_saved)

for l in range(2):
    for k in range(2):
        for i in range(2):
            for j in range(2):
                print(data[j, i, k ,l], end=", ")
            print()
        print("")
    print("+++++++++")
print(data.shape)
print("\n--------------")

write_csv3d(data, "filter")